# Next steps 

Add description

In [19]:
# Import statements
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

# Import the files we need 

In [20]:
VVR_measurements_clean = pd.read_csv('/Users/dionnespaltman/Desktop/V4/VVR_measurements/VVR_measurements_clean.csv', sep=',')
VVR_measurements_total = pd.read_csv('/Users/dionnespaltman/Desktop/V4/VVR_measurements/VVR_measurements.csv', sep=',')

if 'Unnamed: 0' in VVR_measurements_clean.columns:
    VVR_measurements_clean.drop(columns=['Unnamed: 0'], inplace=True)

if 'Unnamed: 0' in VVR_measurements_total.columns:
    VVR_measurements_total.drop(columns=['Unnamed: 0'], inplace=True)

display(VVR_measurements_clean)
display(VVR_measurements_total)

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender
0,5,22.0,34.0,12.0,10.0,26.0,0,1,2019-10-21,2
1,6,16.0,32.0,8.0,8.0,24.0,0,2,2019-10-25,1
2,7,23.0,77.0,11.0,12.0,65.0,1,2,2019-10-28,2
3,8,17.0,32.0,9.0,8.0,24.0,0,1,2019-10-30,1
4,9,16.0,32.0,8.0,8.0,24.0,0,1,2019-10-31,1
...,...,...,...,...,...,...,...,...,...,...
315,328,20.0,35.0,10.0,10.0,27.0,0,3,2022-11-23,2
316,329,18.0,35.0,10.0,8.0,27.0,0,3,2022-11-28,2
317,330,23.0,35.0,11.0,12.0,26.0,0,3,2022-11-30,2
318,331,28.0,38.0,14.0,14.0,30.0,0,3,2022-11-30,2


,ID,Stage,Gender,Age,Date,Location,Condition,Weight,Length,Faintness,Dizziness,Weakness,Lightheadedness,Fear,Tension,Stress,Nervousness,Physical_sum,Psychological_sum
0,5,1.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,4.0,8.0
1,5,2.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0
2,5,3.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0
3,5,4.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0
4,5,5.0,2,33.0,2019-10-21,0,1,74.0,171.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,332,2.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4.0,4.0,12.0
2045,332,4.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,3.0,3.0,3.0,5.0,4.0,14.0
2046,332,5.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,3.0,3.0,4.0,4.0,12.0
2047,332,6.0,2,21.0,2022-12-19,3,2,66.0,173.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,3.0,4.0,9.0


Loading the features we extracted from TS fresh. Here we only have data from 104 people. 

In [21]:
extracted_features = pd.read_pickle('/Users/dionnespaltman/Desktop/V4/extracted_features.pkl')

# Add a new column 'ID' to the DataFrame with values as the index of the rows
extracted_features['ID'] = extracted_features.index

display(extracted_features)

,AU10_c__sum_values,AU10_c__variance,AU10_c__standard_deviation,AU10_c__maximum,AU10_c__minimum,AU10_c__median,AU10_c__mean,AU10_c__mean_abs_change,"AU10_c__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU10_c__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""",...,AU09_r__variance,AU09_r__standard_deviation,AU09_r__maximum,AU09_r__minimum,AU09_r__median,AU09_r__mean,AU09_r__mean_abs_change,"AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""",ID
23,7873.0,0.248750,0.498748,1.0,0.0,1.0,0.535360,0.024685,-0.000017,-0.000013,...,0.109496,0.330902,4.98,0.00,0.00,0.120554,0.019213,-0.000032,-0.000022,23
24,16831.0,0.235917,0.485713,1.0,0.0,1.0,0.618673,0.010881,0.000053,0.000053,...,0.063365,0.251725,2.59,0.00,0.00,0.117537,0.024182,-0.000029,-0.000019,24
25,3506.0,0.168213,0.410138,1.0,0.0,0.0,0.214015,0.011110,-0.000043,-0.000027,...,0.153114,0.391297,4.10,0.00,0.00,0.138912,0.024858,0.000048,0.000033,25
26,867.0,0.046539,0.215729,1.0,0.0,0.0,0.048933,0.003612,-0.000031,-0.000027,...,0.052783,0.229745,3.25,0.00,0.00,0.099096,0.016996,-0.000013,-0.000012,26
27,200.0,0.009414,0.097026,1.0,0.0,0.0,0.009504,0.002281,-0.000005,-0.000002,...,0.779602,0.882951,3.45,-4.43,1.05,0.992115,0.150508,0.000089,0.000095,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,4704.0,0.148649,0.385550,1.0,0.0,0.0,0.181643,0.010967,0.000057,0.000048,...,0.044894,0.211882,3.71,0.00,0.00,0.089742,0.014612,-0.000008,-0.000007,140
142,9082.0,0.238788,0.488660,1.0,0.0,0.0,0.394116,0.020049,0.000013,0.000019,...,0.144935,0.380703,4.70,0.00,0.00,0.133172,0.022507,0.000057,0.000042,142
144,252.0,0.013292,0.115292,1.0,0.0,0.0,0.013474,0.001925,0.000002,0.000001,...,0.123643,0.351629,5.00,0.00,0.00,0.123398,0.018496,-0.000025,-0.000019,144
145,10297.0,0.220715,0.469804,1.0,0.0,0.0,0.328873,0.011115,0.000052,0.000050,...,0.065670,0.256262,3.08,0.00,0.00,0.120352,0.016477,0.000025,0.000020,145


# Merging files 

In [22]:
# Assuming 'ID' is the common column in both DataFrames
merged_df = pd.merge(VVR_measurements_clean, extracted_features, on='ID', how='inner')

# Check the length of the merged DataFrame
print(len(merged_df))
display(merged_df)

104


,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU09_r__sum_values,AU09_r__variance,AU09_r__standard_deviation,AU09_r__maximum,AU09_r__minimum,AU09_r__median,AU09_r__mean,AU09_r__mean_abs_change,"AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min"""
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,1772.86,0.109496,0.330902,4.98,0.00,0.00,0.120554,0.019213,-0.000032,-0.000022
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,3197.59,0.063365,0.251725,2.59,0.00,0.00,0.117537,0.024182,-0.000029,-0.000019
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,2275.65,0.153114,0.391297,4.10,0.00,0.00,0.138912,0.024858,0.000048,0.000033
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,1755.79,0.052783,0.229745,3.25,0.00,0.00,0.099096,0.016996,-0.000013,-0.000012
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,20877.08,0.779602,0.882951,3.45,-4.43,1.05,0.992115,0.150508,0.000089,0.000095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,2324.06,0.044894,0.211882,3.71,0.00,0.00,0.089742,0.014612,-0.000008,-0.000007
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,3068.82,0.144935,0.380703,4.70,0.00,0.00,0.133172,0.022507,0.000057,0.000042
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,2307.91,0.123643,0.351629,5.00,0.00,0.00,0.123398,0.018496,-0.000025,-0.000019
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,3768.23,0.065670,0.256262,3.08,0.00,0.00,0.120352,0.016477,0.000025,0.000020


In [23]:
merged_df.to_csv('/Users/dionnespaltman/Desktop/V4/complete_dataset.csv', index=True)

In [9]:
# Count the number of NaN values in each column of the DataFrame
nan_counts = merged_df.isna().sum()

# Print columns with NaN values
nan_columns = nan_counts[nan_counts > 0]
print("Columns with NaN values:")
print(nan_columns)

Columns with NaN values:
Series([], dtype: int64)


# Get list of action units

In [36]:
import json

# Open the JSON file and load its contents into a list
with open('/Users/dionnespaltman/Desktop/V4/columns_action_units.json', 'r') as f:
    columns_action_units = json.load(f)

print("Loaded columns_action_units from JSON file:", len(columns_action_units))


Loaded columns_action_units from JSON file: 350


# Descriptives 

In [26]:
complete_dataset = pd.read_csv('/Users/dionnespaltman/Desktop/V4/complete_dataset.csv', sep=',')

if 'Unnamed: 0' in complete_dataset.columns:
    complete_dataset.drop(columns=['Unnamed: 0'], inplace=True)

display(complete_dataset)

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU09_r__sum_values,AU09_r__variance,AU09_r__standard_deviation,AU09_r__maximum,AU09_r__minimum,AU09_r__median,AU09_r__mean,AU09_r__mean_abs_change,"AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min"""
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,1772.86,0.109496,0.330902,4.98,0.00,0.00,0.120554,0.019213,-0.000032,-0.000022
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,3197.59,0.063365,0.251725,2.59,0.00,0.00,0.117537,0.024182,-0.000029,-0.000019
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,2275.65,0.153114,0.391297,4.10,0.00,0.00,0.138912,0.024858,0.000048,0.000033
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,1755.79,0.052783,0.229745,3.25,0.00,0.00,0.099096,0.016996,-0.000013,-0.000012
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,20877.08,0.779602,0.882951,3.45,-4.43,1.05,0.992115,0.150508,0.000089,0.000095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,2324.06,0.044894,0.211882,3.71,0.00,0.00,0.089742,0.014612,-0.000008,-0.000007
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,3068.82,0.144935,0.380703,4.70,0.00,0.00,0.133172,0.022507,0.000057,0.000042
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,2307.91,0.123643,0.351629,5.00,0.00,0.00,0.123398,0.018496,-0.000025,-0.000019
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,3768.23,0.065670,0.256262,3.08,0.00,0.00,0.120352,0.016477,0.000025,0.000020


# Get the range of dates 

In [28]:
# Find the first and last dates
first_date = complete_dataset['Date'].min()
last_date = complete_dataset['Date'].max()

print("First date:", first_date)
print("Last date:", last_date)

First date: 2020-01-22
Last date: 2021-12-05


# Get the counts of condition

In [29]:
# Count of unique values in the 'Condition' column in clean_donor_info
complete_dataset_condition_count = complete_dataset['Condition'].value_counts()

# Display the counts
print("Count of Condition in the complete dataset")
print(complete_dataset_condition_count)

Count of Condition in the complete dataset
1    56
3    30
2    18
Name: Condition, dtype: int64


# Get the counts of VVR group

In [30]:
complete_dataset_VVR_count = complete_dataset['VVR_group'].value_counts()

# Display the counts
print("Count of VVR_group in the complete dataset")
print(complete_dataset_VVR_count)

Count of VVR_group in the complete dataset
0    81
1    23
Name: VVR_group, dtype: int64


# Distribution of gender 

In [31]:
complete_dataset_gender_count = complete_dataset['Gender'].value_counts()

# Display the counts
print("Count of gender in the complete dataset")
print(complete_dataset_gender_count)

Count of gender in the complete dataset
2    65
1    39
Name: Gender, dtype: int64


# Reducing features with TS Fresh 

https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_selection.html

In [43]:
# Check for missing values in any columns
missing_values_per_column = complete_dataset.isnull().any()

# Get the columns with missing values
columns_with_missing_values = missing_values_per_column[missing_values_per_column].index.tolist()

print("Columns with missing values:", columns_with_missing_values)

Columns with missing values: []


In [33]:
display(complete_dataset)
print("Loaded columns_action_units from JSON file:", len(columns_action_units))

,ID,Sum_12,Sum_4567,VVR_1,VVR_2,Sum_456,VVR_group,Condition,Date,Gender,...,AU09_r__sum_values,AU09_r__variance,AU09_r__standard_deviation,AU09_r__maximum,AU09_r__minimum,AU09_r__median,AU09_r__mean,AU09_r__mean_abs_change,"AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU09_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min"""
0,23,24.0,37.0,13.0,11.0,27.0,0,2,2020-08-01,2,...,1772.86,0.109496,0.330902,4.98,0.00,0.00,0.120554,0.019213,-0.000032,-0.000022
1,24,23.0,37.0,12.0,11.0,28.0,0,2,2020-01-22,2,...,3197.59,0.063365,0.251725,2.59,0.00,0.00,0.117537,0.024182,-0.000029,-0.000019
2,25,28.0,44.0,16.0,12.0,33.0,1,2,2020-05-02,2,...,2275.65,0.153114,0.391297,4.10,0.00,0.00,0.138912,0.024858,0.000048,0.000033
3,26,30.0,37.0,15.0,15.0,29.0,0,1,2020-06-02,1,...,1755.79,0.052783,0.229745,3.25,0.00,0.00,0.099096,0.016996,-0.000013,-0.000012
4,27,22.0,39.0,11.0,11.0,31.0,0,2,2020-06-02,1,...,20877.08,0.779602,0.882951,3.45,-4.43,1.05,0.992115,0.150508,0.000089,0.000095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,140,16.0,32.0,8.0,8.0,24.0,0,3,2021-05-26,2,...,2324.06,0.044894,0.211882,3.71,0.00,0.00,0.089742,0.014612,-0.000008,-0.000007
100,142,20.0,34.0,11.0,9.0,26.0,0,3,2021-05-31,1,...,3068.82,0.144935,0.380703,4.70,0.00,0.00,0.133172,0.022507,0.000057,0.000042
101,144,24.0,35.0,12.0,12.0,27.0,0,3,2021-01-06,1,...,2307.91,0.123643,0.351629,5.00,0.00,0.00,0.123398,0.018496,-0.000025,-0.000019
102,145,20.0,37.0,11.0,9.0,28.0,0,1,2021-02-06,2,...,3768.23,0.065670,0.256262,3.08,0.00,0.00,0.120352,0.016477,0.000025,0.000020


No features are seen as relevant. That could have something to do with the fact that we have a lot of features and very little data. 


In [49]:
from tsfresh.feature_selection.relevance import calculate_relevance_table

columns_to_drop = ['ID', 'Sum_12', 'Sum_4567', 'VVR_1', 'VVR_2', 'Sum_456', 'VVR_group', 'Condition', 'Date', 'Gender']

X = complete_dataset.drop(columns=columns_to_drop)
y = complete_dataset['VVR_group']

# Calculate relevance table
relevance_table = calculate_relevance_table(X, y, ml_task='classification', multiclass=False)

print("Relevance table:")
display(relevance_table)


Relevance table:


,feature,type,p_value,relevant
feature,,,,
AU15_r__median,AU15_r__median,real,0.011627,False
AU07_c__mean,AU07_c__mean,real,0.022635,False
"AU17_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""min""","AU17_r__agg_linear_trend__attr_""slope""__chunk_...",real,0.024563,False
"AU17_r__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU17_r__agg_linear_trend__attr_""slope""__chunk_...",real,0.027172,False
AU26_r__median,AU26_r__median,real,0.028800,False
...,...,...,...,...
"AU07_c__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU07_c__agg_linear_trend__attr_""slope""__chunk_...",real,0.993749,False
"AU23_c__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""max""","AU23_c__agg_linear_trend__attr_""slope""__chunk_...",real,0.993749,False
AU09_r__maximum,AU09_r__maximum,real,0.996875,False


In [50]:
# Count the number of relevant features
relevant_count = relevance_table['relevant'].value_counts()

print("Count of relevant features:")
print(relevant_count)


Count of relevant features:
False    350
Name: relevant, dtype: int64
